# Kundur Two Areas Sinusoidal Steady-State Co-simulation

### Import Libraries

In [ ]:
import subprocess, sys, os
import urllib.request

dpsim_root_dir = subprocess.Popen(['git', 'rev-parse', '--show-toplevel'], stdout=subprocess.PIPE).communicate()[0].rstrip().decode('utf-8')
sys.path.insert(0, os.path.join(dpsim_root_dir, 'python/src/dpsim/'))
sys.path.insert(0, os.path.join(dpsim_root_dir, 'build'))

import matpower
import dpsimpy
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import urllib.request
import matplotlib.pyplot as plt
# %matplotlib widget

### Get simulation data

In [ ]:
if not os.path.exists('Kundur2Areas-data'):
    os.mkdir('Kundur2Areas-data')

url_static = 'https://github.com/martinmoraga/dpsim_data/raw/main/Kundur2Areas/matpower/Kundur2Areas.mat'
url_dynamic = 'https://github.com/martinmoraga/dpsim_data/raw/main/Kundur2Areas/matpower/Kundur2Areas_dyn.mat'
local_file_static = './Kundur2Areas-data/Kundur2Areas.mat'
local_file_dynamic = './Kundur2Areas-data/Kundur2Areas_dyn.mat'
urllib.request.urlretrieve(url_static, local_file_static)
urllib.request.urlretrieve(url_dynamic, local_file_dynamic)

time_step = 1e-6
t_f = 0.99

### 1. Powerflow for initialization

In [ ]:
sim_name_pf = 'Kundur2Areas_PF'
dpsimpy.Logger.set_log_dir('logs/' + sim_name_pf)

# read and create dpsim topology
mpc_reader_pf = matpower.Reader(mpc_file_path=local_file_static, mpc_name='Kundur2Areas')
mpc_reader_pf.load_mpc(domain=matpower.Domain.PF)
system_pf = mpc_reader_pf.system

# log results
logger = dpsimpy.Logger(sim_name_pf)
for node in system_pf.nodes:
    logger.log_attribute(node.name()+'.V', 'v', node)
    logger.log_attribute(node.name()+'.S', 's', node)

# Parametrize and run simulation
sim_pf = dpsimpy.Simulation(sim_name_pf, dpsimpy.LogLevel.info)
sim_pf.set_system(system_pf)
sim_pf.set_time_step(1)
sim_pf.set_final_time(0.1)
sim_pf.set_domain(dpsimpy.Domain.SP)
sim_pf.set_solver(dpsimpy.Solver.NRP)
sim_pf.do_init_from_nodes_and_terminals(False)
sim_pf.set_solver_component_behaviour(dpsimpy.SolverBehaviour.Initialization)
sim_pf.add_logger(logger)
sim_pf.run()

In [ ]:
#system_pf

### 2. Steady-state, Monolithic simulation

#### Declare some functions

In [ ]:
def Kundur2Areas_SS(domain="SP"):
    if domain=="SP":
        matpower_domain = matpower.Domain.SP
        dpsim_domain = dpsimpy.Domain.SP
    elif domain=="DP":
        matpower_domain = matpower.Domain.DP
        dpsim_domain = dpsimpy.Domain.DP
    else:
        domain="EMT"
        matpower_domain = matpower.Domain.EMT
        dpsim_domain = dpsimpy.Domain.EMT

    sim_name_ss = domain + "_Kundur2Areas_SS"        
    dpsimpy.Logger.set_log_dir('logs/' + sim_name_ss)

    mpc_reader_dyn = matpower.Reader(mpc_file_path=local_file_static, mpc_name='Kundur2Areas',
                                     mpc_dyn_file_path=local_file_dynamic, mpc_dyn_name='Kundur2Areas_dyn')
    mpc_reader_dyn.create_dpsim_objects(domain=matpower_domain, frequency=60, 
                                     with_avr=False, with_tg=False, with_pss=False)

    # create dpsim topology
    mpc_reader_dyn.create_dpsim_topology()

    #initialize node voltages using pf results
    system_dyn = mpc_reader_dyn.system
    system_dyn.init_with_powerflow(system_pf, dpsim_domain)

    # log results
    logger = dpsimpy.Logger(sim_name_ss)
    for node in system_dyn.nodes:
        logger.log_attribute(node.name()+'.V', 'v', node)
    # for gen_name in ["Gen_N1", "Gen_N2", "Gen_N3", "Gen_N3", "Gen_N4"]:
    #     logger.log_attribute(gen_name+".Pe", 'Te', mpc_reader_dyn.dpsimpy_comp_dict[gen_name][0])

    # line3_7_8 = system_dyn.component("line3_7-8")
    # print(line3_7_8[0])
    logger.log_attribute("line5_8-9.I", 'i_intf', system_dyn.component("line5_8-9"))
    logger.log_attribute("line6_8-9.I", 'i_intf', system_dyn.component("line6_8-9"))

    # Parametrize and run simulation
    sim = dpsimpy.Simulation(sim_name_ss, dpsimpy.LogLevel.info)
    sim.set_system(system_dyn)
    if domain=="SP":
        sim.set_time_step(1e-3)
    else:
        sim.set_time_step(time_step)
    sim.set_final_time(t_f)
    sim.set_domain(dpsim_domain)
    sim.set_solver(dpsimpy.Solver.MNA)
    sim.set_direct_solver_implementation(dpsimpy.DirectLinearSolverImpl.KLU)
    sim.do_init_from_nodes_and_terminals(True)
    sim.add_logger(logger)
    sim.do_system_matrix_recomputation(True)

    sim.run()
    
    return sim_name_ss

#### Dpsim Simulations

In [ ]:
sim_name_ss_emt = Kundur2Areas_SS(domain="EMT")

### Split topologie at node 8

In [ ]:
# H_v = np.round(np.logspace(-6,-3,10), 6)

H_v = np.array([2e-6, 3e-6, 6e-6, 1e-5, 2e-5, 3.6e-5, 6.6e-5, 1.2e-4, 2.2e-4, 4e-4])
print(H_v)

H = 6e-6

interp = 'zoh'

k_map = {'zoh': 0, 'linear': 1}
k = k_map[interp]

cosim_config = {
    "number_topologies": 2,
    "nodes": [["N1", "N2", "N5", "N6", "N7", "N8"], ["N3", "N4", "N11", "N10", "N9", "N8"]],
    "eq_component": ["VS", "CS"],
    "split_node": "N8",
    "sim_names": ["Cosim_Kundur2Areas_SS_S1", "Cosim_Kundur2Areas_SS_S2"]
}

#### Create topologies for cosimulation

In [ ]:
sim_name_ss = "Cosim_Kundur2Areas_SS"        
dpsimpy.Logger.set_log_dir('logs/' + sim_name_ss)

# load dynamic topology
mpc_reader_dyn = matpower.Reader(mpc_file_path=local_file_static, mpc_name='Kundur2Areas',
                                 mpc_dyn_file_path=local_file_dynamic, mpc_dyn_name='Kundur2Areas_dyn')
mpc_reader_dyn.create_dpsim_objects(domain=matpower.Domain.EMT, frequency=60, 
                                 with_avr=False, with_tg=False, with_pss=False)

# create dpsim topology
mpc_reader_dyn.create_dpsim_topology()

#initialize node voltages using pf results
system_dyn = mpc_reader_dyn.system
system_dyn.init_with_powerflow(system_pf, dpsimpy.Domain.EMT)

#create topologies for cosimulation
[sys_topo_1_ss, sys_topo_2_ss] = mpc_reader_dyn.create_cosim_topologies(cosim_config)

#### Prepare 2nd sub topology

In [ ]:
sim_name_ss_s2 = "Cosim_Kundur2Areas_SS_S2_" + interp + '_' + str(H)
dpsimpy.Logger.set_log_dir('logs/' + sim_name_ss_s2)

# set initial reference current of current source
init_current = sim_pf.get_idobj_attr("line5_8-9", 'current_vector').get()[1][0] + sim_pf.get_idobj_attr("line6_8-9", 'current_vector').get()[1][0]
init_current = dpsimpy.Math.single_phase_variable_to_three_phase(init_current * np.sqrt(3))
sys_topo_2_ss.component("CS_N8").set_parameters(-init_current, 0)

# log results
logger = dpsimpy.Logger(sim_name_ss_s2)
for node in sys_topo_2_ss.nodes:
    logger.log_attribute(node.name()+'.V', 'v', node)
#for gen_name in ["Gen_N1", "Gen_N2"]:
#    logger.log_attribute(gen_name+".Pe", 'Te', mpc_reader_dyn.dpsimpy_comp_dict[gen_name][0])

logger.log_attribute(sys_topo_2_ss.component("CS_N8").name()+'.I', 'i_intf', sys_topo_2_ss.component("CS_N8")) 
logger.log_attribute(sys_topo_2_ss.component("CS_N8").name()+'.V', 'v_intf', sys_topo_2_ss.component("CS_N8")) 


# Parametrize and run simulation
sim2 = dpsimpy.Simulation(sim_name_ss_s2, dpsimpy.LogLevel.info)
sim2.set_system(sys_topo_2_ss)
sim2.set_time_step(time_step)
sim2.set_final_time(t_f)
sim2.set_domain(dpsimpy.Domain.EMT)
sim2.set_solver(dpsimpy.Solver.MNA)
sim2.set_direct_solver_implementation(dpsimpy.DirectLinearSolverImpl.KLU)
sim2.do_init_from_nodes_and_terminals(True)
sim2.add_logger(logger)
sim2.do_system_matrix_recomputation(True)

sim2.start()
y_2_0 = sim2.get_idobj_attr("CS_N8", "v_intf").get()

#### Prepare 1st sub topology

In [ ]:
sim_name_ss_s1 = "Cosim_Kundur2Areas_SS_S1_" + interp + '_' + str(H)      
dpsimpy.Logger.set_log_dir('logs/' + sim_name_ss_s1)

# set initial current of source current
init_current = sim_pf.get_idobj_attr("line3_7-8", 'current_vector').get()[0][0] + sim_pf.get_idobj_attr("line4_7-8", 'current_vector').get()[0][0]
init_current = dpsimpy.Math.single_phase_variable_to_three_phase(init_current)
sys_topo_1_ss.component("VS_N8").set_intf_current(-np.real(init_current) * np.sqrt(3) * dpsimpy.RMS3PH_TO_PEAK1PH)

# set parameters of source current
init_voltage = sim_pf.get_idobj_attr("N8", 'v').get()[0]
sys_topo_1_ss.component("VS_N8").set_parameters(dpsimpy.Math.single_phase_variable_to_three_phase(init_voltage), 0)

# log results
logger = dpsimpy.Logger(sim_name_ss_s1)
for node in sys_topo_1_ss.nodes:
    logger.log_attribute(node.name()+'.V', 'v', node)
#for gen_name in ["Gen_N1", "Gen_N2"]:
#    logger.log_attribute(gen_name+".Pe", 'Te', mpc_reader_dyn.dpsimpy_comp_dict[gen_name][0])

logger.log_attribute(sys_topo_1_ss.component("VS_N8").name()+'.I', 'i_intf', sys_topo_1_ss.component("VS_N8")) 
logger.log_attribute(sys_topo_1_ss.component("VS_N8").name()+'.V', 'v_intf', sys_topo_1_ss.component("VS_N8")) 

# Parametrize and run simulation
sim1 = dpsimpy.Simulation(sim_name_ss_s1, dpsimpy.LogLevel.info)
sim1.set_system(sys_topo_1_ss)
sim1.set_time_step(time_step)
sim1.set_final_time(t_f)
sim1.set_domain(dpsimpy.Domain.EMT)
sim1.set_solver(dpsimpy.Solver.MNA)
sim1.set_direct_solver_implementation(dpsimpy.DirectLinearSolverImpl.KLU)
sim1.do_init_from_nodes_and_terminals(True)
sim1.add_logger(logger)
sim1.do_system_matrix_recomputation(True)

sim1.start()
y_1_0 = sim1.get_idobj_attr("VS_N8", "i_intf").get() * dpsimpy.PEAK1PH_TO_RMS3PH

#### Execute cosimulation

In [ ]:
t_0 = 0
t_k_v = []
j_v = []
t_k = 0.0
y_1 = y_1_0

print(y_1)

n = int(round((t_f - t_0) / time_step))
t = np.around(np.linspace(t_0, t_f, n + 1), 16)

m = int(np.around(H/time_step))
print("m = " + str(m))

N = int(round((t_f - t_0) / H))
print("N = " + str(N))

t_m = np.around(np.linspace(t_0, t_f, N + 1), 16)

# We have to assume the trajectory of y_2 extending its initial value, since we have no prior information
# y_1_m_prev = np.tile(y_1_0, np.max([k, m]))
y_1_m_prev = np.tile(y_1_0, np.min([k+1, m]))

for i in range(0, N):   
    y_1_prev = y_1_m_prev[:,-1]
    t_m_i = t[m*i + 1: m*(i+1) + 1]

    if interp == 'zoh':
        u_2_m = np.tile(y_1_prev, (m,1)).T
    elif interp == 'linear':
        # t_poly = np.array([i*H-H, i*H])
        # f_u_2 = np.polynomial.polynomial.polyfit(t_poly, y_1_m_prev[:,-2:].T, 1)
        # u_2_m = np.polynomial.polynomial.polyval(t_m_i, f_u_2)

        u_2_m = np.zeros((len(y_1_0), m))
        for ind_2 in range(0, len(y_1_0)):
            f_u_2 = np.poly1d(np.polyfit([i*H-H, i*H], y_1_m_prev[ind_2, -2:], 1))
            u_2_m[ind_2,:] = f_u_2(t_m_i)

    y_1_m = np.zeros((len(y_1_0), m))
    
    for j in range(0, m):
        # Switch to S_2                
        u_2 = u_2_m[:,j]
        # print(u_2)
        sim2.get_idobj_attr("CS_N8", "I_ref").set(u_2)
        
        sim2.next()
        y_2 = sim2.get_idobj_attr("CS_N8", "v_intf").get() * dpsimpy.PEAK1PH_TO_RMS3PH

        u_1 = y_2
        # print(u_1)
        
        sim1.get_idobj_attr("VS_N8", "V_ref").set(u_1)
        t_k = sim1.next()
        y_1 = sim1.get_idobj_attr("VS_N8", "i_intf").get() * dpsimpy.PEAK1PH_TO_RMS3PH
        # print(y_1)
        
        y_1_m[:,j] = y_1.flatten()
        # j += 1
        t_k_v.append(t_k)

    # Option 1: Take values at macrosteps as samples for extrapolation
    y_1_m_prev = np.hstack((y_1_m_prev, y_1_m[:,-1][np.newaxis].T))

    # Option 2: Take values at microsteps as samples for extrapolation
    # y_1_m_prev = y_1_m
    
    # j_v.append(j)

    if t_k > t_f:
        print(t_k)
        print("i: " + str(i) + ", expected: " + str(N-1))
        print("j: " + str(j) + ", expected: " + str(m-1))
        break

sim1.stop()
sim2.stop()

### Compare steady state results

In [ ]:
from villas.dataprocessing.timeseries import TimeSeries as ts
import villas.dataprocessing.plottools as pt

dpsim_result_file_ss = 'logs/' + sim_name_ss_emt + '/' + sim_name_ss_emt + '.csv'
ts_dpsim_ss_emt = read_timeseries_csv(dpsim_result_file_ss)

dpsim_result_file_ss_s1 = 'logs/' + sim_name_ss_s1 + '/' + sim_name_ss_s1 + '.csv'
ts_dpsim_ss_s1_emt = read_timeseries_csv(dpsim_result_file_ss_s1)

dpsim_result_file_ss_s2 = 'logs/' + sim_name_ss_s2 + '/' + sim_name_ss_s2 + '.csv'
ts_dpsim_ss_s2_emt = read_timeseries_csv(dpsim_result_file_ss_s2)

### Define plot functions

In [ ]:
timestep_common = H
t_begin = 0.0
t_end = t_f
begin_idx = int(t_begin/timestep_common)
end_idx= int(t_end/timestep_common)
time = np.linspace(t_begin, t_end, num=end_idx-begin_idx)

#plot parameters
width = 8
height = 4

def plot_node_volt_abs_cosim(varname_dpsim, 
                       ts_dpsim_emt, ts_dpsim_emt_s1, ts_dpsim_emt_s2, nominal_voltage, timestep_common=1e-3, y_lim=False):
    
    #convert dpsim voltage to magnitude value and per-unit for comparison with psat
    dpsim_emt_values = ts_dpsim_emt[varname_dpsim+"_0"].interpolate(timestep_common).values[begin_idx:end_idx] / nominal_voltage * np.sqrt(3/2)
    dpsim_emt_values_s1 = ts_dpsim_emt_s1[varname_dpsim+"_0"].interpolate(timestep_common).values[begin_idx:end_idx] / nominal_voltage * np.sqrt(3/2)
    dpsim_emt_values_s2 = ts_dpsim_emt_s2[varname_dpsim+"_0"].interpolate(timestep_common).values[begin_idx:end_idx] / nominal_voltage * np.sqrt(3/2)
    
    plt.figure(figsize=(width, height))
    # plt.subplot(1, 2, 1)
    plt.plot(time, dpsim_emt_values, label='EMT - DPsim - Monolithic')
    plt.plot(time, dpsim_emt_values_s1, label='EMT - DPsim - S1')
    plt.plot(time, dpsim_emt_values_s2, '--', label='EMT - DPsim - S2')
    plt.legend(loc='lower right')
    plt.xlabel('time (s)')
    plt.grid()
    #plt.ylim([1.058, 1.062])
    # plt.xlim([0.95, 1.2])
    # plt.xlim([0, 0.08])
    plt.xlabel("time (s)")
    plt.ylabel(varname_dpsim + " (pu)")
    if (y_lim):
        plt.ylim(y_lim)
    
    # plt.subplot(1, 2, 2)
    # plt.plot(time, dpsim_emt_values, ':', label='EMT - DPsim')
    # plt.legend(loc='lower right')
    # plt.xlabel('time (s)')
    # plt.grid()
    # #plt.ylim([1.058, 1.062])
    # # plt.xlim([2.9, 3.1])
    # plt.xlabel("time (s)")
    # plt.ylabel(varname_dpsim + " (pu)")
    # if (y_lim):
    #     plt.ylim(y_lim)
        
    plt.show()
    
    #calculate RMSE
    #rmse = np.sqrt(((dpsim_sp_values_abs_pu - psat_values) ** 2).mean())
    #print('RMSE {:s}  = {:.6f} (pu), which is {:.3f}% of the nominal value = {:.3f} (pu) '.format(varname_dpsim, rmse, rmse/1.0*100, 1.0))
    
# def plot_elec_power(varname_dpsim, varname_digSilent, 
#                     ts_dpsim_emt, ts_dpsim_dp, ts_digSilent, timestep_common=1e-3, y_lim=False):
    
#     #convert dpsim voltage to magnitude value and per-unit for comparison with psat
#     dpsim_dp_values = ts_dpsim_dp[varname_dpsim].interpolate(timestep_common).values[begin_idx:end_idx] * 9
#     dpsim_emt_values = ts_dpsim_emt[varname_dpsim].interpolate(timestep_common).values[begin_idx:end_idx] * 9
#     digSilent_values = ts_digSilent[varname_digSilent].interpolate(timestep_common).values[begin_idx:end_idx] / 100
    
#     plt.figure(figsize=(width, height))
#     plt.plot(time, dpsim_dp_values, label='DP - DPsim')
#     plt.plot(time, dpsim_emt_values, '--', label='EMT - DPsim')
#     plt.plot(time, digSilent_values, '--', label='EMT - DigSilent')
#     plt.legend(loc='lower right')
#     plt.xlabel('time (s)')
#     plt.grid()
#     #plt.ylim([6.7, 7.1])
#     plt.xlim([t_begin, t_end])
#     plt.xlabel("time (s)")
#     plt.ylabel(varname_dpsim + " (pu)")
#     if (y_lim):
#         plt.ylim(y_lim)
#     plt.show()
    
    #calculate RMSE
    #rmse = np.linalg.norm(dpsim_sp_values - psat_values) / np.sqrt(len(psat_values))
    #print('RMSE {:s}  = {:.6f} (pu), which is {:.3f}% of the nominal value = ~{:.3f} (pu) '.format(varname_dpsim, rmse, rmse/7.0*100, 7.0))

    #return rmse

#### Inteface node

In [ ]:
#"""
varname_dpsim = 'N8.V'
nominal_voltage = 230000
plot_node_volt_abs_cosim(varname_dpsim, ts_dpsim_ss_emt, ts_dpsim_ss_s1_emt, ts_dpsim_ss_s2_emt, nominal_voltage, timestep_common)
#"""

### Plot interface current

In [ ]:
#plot parameters
width = 8
height = 4

current_1_8_9 = ts_dpsim_ss_emt["line5_8-9.I_0"].interpolate(timestep_common).values[begin_idx:end_idx]
current_2_8_9 = ts_dpsim_ss_emt["line6_8-9.I_0"].interpolate(timestep_common).values[begin_idx:end_idx]
dpsim_emt_values = current_1_8_9 + current_2_8_9
dpsim_emt_values_s1 = ts_dpsim_ss_s1_emt["VS_N8.I_0"].interpolate(timestep_common).values[begin_idx:end_idx]
dpsim_emt_values_s2 = ts_dpsim_ss_s2_emt["CS_N8.I_0"].interpolate(timestep_common).values[begin_idx:end_idx]
    
plt.figure(figsize=(width, height))
#plt.subplot(1, 2, 1)
plt.plot(time, dpsim_emt_values, label='EMT - Monolithic')
plt.plot(time, dpsim_emt_values_s1, ':', label='EMT - DPsim - S1')
plt.plot(time, dpsim_emt_values_s2, '--', label='EMT - DPsim - S2')
plt.legend(loc='lower right')
plt.xlabel('time (s)')
plt.grid()
#plt.ylim([1.058, 1.062])
plt.xlim([0, 0.08])
plt.xlabel("time (s)")
#plt.ylabel(varname_digSilent + " (pu)")